# LAB 5b:  Deploy and predict with Keras model on Cloud AI Platform.

**Learning Objectives**

1. Setup up the environment
1. Deploy trained Keras model to Cloud AI Platform
1. Online predict from model on Cloud AI Platform
1. Batch predict from model on Cloud AI Platform


## Introduction 
In this notebook, we'll deploying our Keras model to Cloud AI Platform and creating predictions.

We will set up the environment, deploy a trained Keras model to Cloud AI Platform, online predict from deployed model on Cloud AI Platform, and batch predict from deployed model on Cloud AI Platform.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/5b_deploy_keras_ai_platform_babyweight.ipynb).

## Set up environment variables and load necessary libraries

Import necessary libraries.

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [1]:
import os

### Set environment variables.

Set environment variables so that we can use them throughout the entire lab. We will be using our project name for our bucket, so you only need to change your project and region.

In [2]:
%%bash
PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: michaelabel-gcp-training


In [8]:
# Change these to try this notebook out
PROJECT = "your-project-name-here"  # Replace with your PROJECT
BUCKET = PROJECT  # defaults to PROJECT
REGION = "us-central1"  # Replace with your REGION

In [9]:
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.1"

In [6]:
%%bash
gcloud config set compute/region $REGION
gcloud config set ai_platform/region global

Updated property [compute/region].
Updated property [ai_platform/region].

## Check our trained model files

Let's check the directory structure of our outputs of our trained model in folder we exported the model to in our last [lab](../solutions/10_train_keras_ai_platform_babyweight.ipynb). We'll want to deploy the saved_model.pb within the timestamped directory as well as the variable values in the variables folder. Therefore, we need the path of the timestamped directory so that everything within it can be found by Cloud AI Platform's model deployment service.

In [10]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model

gs://michaelabel-gcp-training-ml/babyweight/trained_model/
gs://michaelabel-gcp-training-ml/babyweight/trained_model/20200428154223/
gs://michaelabel-gcp-training-ml/babyweight/trained_model/20200428221424/
gs://michaelabel-gcp-training-ml/babyweight/trained_model/20200428230409/
gs://michaelabel-gcp-training-ml/babyweight/trained_model/checkpoints/


In [11]:
%%bash
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/babyweight/trained_model/2* \
                 | tail -1)
gsutil ls ${MODEL_LOCATION}

gs://michaelabel-gcp-training-ml/babyweight/trained_model/20200428230409/
gs://michaelabel-gcp-training-ml/babyweight/trained_model/20200428230409/saved_model.pb
gs://michaelabel-gcp-training-ml/babyweight/trained_model/20200428230409/assets/
gs://michaelabel-gcp-training-ml/babyweight/trained_model/20200428230409/variables/


## Lab Task #2: Deploy trained model

Deploying the trained model to act as a REST web service is a simple gcloud call.

In [12]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/babyweight/trained_model/2* \
                 | tail -1 | tr -d '[:space:]')
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION"
# gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
# gcloud ai-platform models delete ${MODEL_NAME}
gcloud ai-platform models create ${MODEL_NAME} --regions ${REGION}
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model=${MODEL_NAME} \
    --origin=${MODEL_LOCATION} \
    --runtime-version=2.1 \
    --python-version=3.7

Deleting and deploying babyweight ml_on_gcp from gs://michaelabel-gcp-training-ml/babyweight/trained_model/20200428230409/


Created ml engine model [projects/michaelabel-gcp-training/models/babyweight].
Creating version (this might take a few minutes)......
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................done.


## Use model to make online prediction.

### Python API

We can use the Python API to send a JSON request to the endpoint of the service to make it predict a baby's weight. The order of the responses are the order of the instances.

In [13]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = "babyweight"
MODEL_VERSION = "ml_on_gcp"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict" \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {"Authorization": "Bearer " + token }
data = {
  "instances": [
    {
      "is_male": "True",
      "mother_age": 26.0,
      "plurality": "Single(1)",
      "gestation_weeks": 39
    },
    {
      "is_male": "False",
      "mother_age": 29.0,
      "plurality": "Single(1)",
      "gestation_weeks": 38
    },
    {
      "is_male": "True",
      "mother_age": 26.0,
      "plurality": "Triplets(3)",
      "gestation_weeks": 39
    },
    {
      "is_male": "Unknown",
      "mother_age": 29.0,
      "plurality": "Multiple(2+)",
      "gestation_weeks": 38
    },
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"predictions": [{"weight": [6.434544563293457]}, {"weight": [6.527022361755371]}, {"weight": [3.9038546085357666]}, {"weight": [6.076201438903809]}]}'


The predictions for the four instances were: 5.33, 6.09, 2.50, and 5.86 pounds respectively when I ran it (your results might be different).

### gcloud shell API

Instead we could use the gcloud shell API. Create a newline delimited JSON file with one instance per line and submit using gcloud.

In [14]:
%%writefile inputs.json
{"is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}
{"is_male": "False", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}

Writing inputs.json


Now call `gcloud ai-platform predict` using the JSON we just created and point to our deployed `model` and `version`.

In [15]:
%%bash
gcloud ai-platform predict \
    --model=babyweight \
    --json-instances=inputs.json \
    --version=ml_on_gcp

WEIGHT
[6.434544563293457]
[6.186980247497559]


## Use model to make batch prediction.

Batch prediction is commonly used when you have thousands to millions of predictions. It will create an actual Cloud AI Platform job for prediction.

In [16]:
%%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ai-platform jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
    --data-format=TEXT \
    --region ${REGION} \
    --input-paths=$INPUT \
    --output-path=$OUTPUT \
    --model=babyweight \
    --version=ml_on_gcp

jobId: babypred_200429_142714
state: QUEUED


Copying file://inputs.json [Content-Type=application/json]...
/ [1 files][  179.0 B/  179.0 B]                                                
Operation completed over 1 objects/179.0 B.                                      
CommandException: 1 files/objects could not be removed.
Job [babypred_200429_142714] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe babypred_200429_142714

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs babypred_200429_142714


## Lab Summary:
In this lab, we set up the environment, deployed a trained Keras model to Cloud AI Platform, online predicted from deployed model on Cloud AI Platform, and batch predicted from deployed model on Cloud AI Platform.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License